In [22]:
def make_prediction(poke1, poke2):
    try:
        import joblib
        import pandas as pd
        import numpy as np
        model_file_name = "svm_best_model.joblib"

        loaded_model = joblib.load(open(model_file_name, 'rb'))

        pokemon_data = pd.read_csv('pokemon.csv', encoding='latin1')	# Ensures special characters are read correctly
        # Normalize numeric values
        names = ['HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
        # Equivalent to using MinMaxScaler()
        for col in names:
            pokemon_data[col] = (pokemon_data[col] - pokemon_data[col].min()) / (pokemon_data[col].max() - pokemon_data[col].min()) 

        # Performs one hot encoding on the 'Type 1' column
        pokemon_data = pd.get_dummies(pokemon_data, columns=['Type 1', ], dtype=int)

        #Dropping Features and encoding True/False
        pokemon_data = pokemon_data.drop('Type 2', axis=1) #Dropping type2 to reduce dimensionality of the data
        pokemon_data = pokemon_data.drop('Generation', axis=1)
        pokemon_data = pokemon_data.drop('#', axis=1)

        # To convert 'Legendary' column from boolean to integer type
        pokemon_data['Legendary'] = pokemon_data['Legendary'].astype(int)

        if poke1 in pokemon_data['Name'].values and poke2 in pokemon_data['Name'].values and poke1 != poke2:
            pokemon1 = pokemon_data[pokemon_data['Name'] == poke1].iloc[0]
            pokemon2 = pokemon_data[pokemon_data['Name'] == poke2].iloc[0]

            # Drop the 'Name' column before prediction, then combine the two
            inputs = pd.concat([pokemon1.drop('Name'), pokemon2.drop('Name')]).values.reshape(1, -1)

            prediction = loaded_model.predict(inputs)

            info = ''

            if (prediction == 0):
                info = poke1
            else:
                info = poke2
        
            final_info = "The winner is {}".format(info)
            return final_info

        
        else:
            final_info = 'These Pokemon are not in the dataset. Please check your spelling and check that the first letter is capitalized.'
            return final_info
    except Exception as e:
        return f"An error occurred: {str(e)}"

#Test
print(make_prediction('Pikachu', 'Charmander'))

The winner is Charmander


c:\Users\moule\miniconda3\envs\ecs171\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


In [ ]:
import gradio as gr

headline = "Pokemon Combat Prediction"

iface = gr.Interface(fn=make_prediction,inputs = ["text", "text"], outputs="text", title=headline)
iface.launch(debug=True, share=True, inline=False)

IMPORTANT: You are using gradio version 3.24.1, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7864

To create a public link, set `share=True` in `launch()`.
Keyboard interruption in main thread... closing server.
